# Generative Adversarial Network

In this notebook, we'll be building a generative adversarial network (GAN) trained on the MNIST dataset. Using this network, we will be generated new digits!

The idea behind GANs is that you have two networks, a generator $G$ and a discriminator $D$, competing against each other. The generator makes fake data to pass to the discriminator. The discriminator also sees real data and predicts if the data it's received is real or fake. The generator is trained to fool the discriminator, it wants to output data that looks _as close as possible_ to real data. And the discriminator is trained to figure out which data is real and which is fake. What ends up happening is that the generator learns to make data that is indistiguishable from real data to the discriminator.

## Generator network

Here we'll build the generator network. We will use a leaky ReLU to allow gradients to flow backwards through the layer unimpeded. A leaky ReLU is like a normal ReLU, except that there is a small non-zero output for negative input values.

#### Leaky ReLU
Typically, a parameter `alpha` sets the magnitude of the output for negative values. So, the output for negative input (`x`) values is `alpha*x`, and the output for positive `x` is `x`:
$$
f(x) = max(\alpha * x, x)
$$
For implementing the leaky relu, you could use the following resource https://keras.io/layers/advanced-activations/

#### Tanh Output
The generator has been found to perform the best with $tanh$ for the generator output. This means that we'll have to rescale the MNIST images to be between -1 and 1, instead of the usual 0 and 1 range. 

## Discriminator

The discriminator network is almost exactly the same as the generator network, except that we're using a sigmoid output layer.

## Build network

Now we're are going to build the generator and discrimator networks.

We'll create the generator, `generator(input_z, input_size)`. This builds the generator with the appropriate input and output sizes.

Then the discriminators. We'll build two of them, one for real data and one for fake data. Since we want the weights to be the same for both real and fake data, we need to reuse the variables. For the fake data, we're getting it from the generator as `g_model`. So the real data discriminator is `discriminator(input_real)` while the fake discriminator is `discriminator(g_model, reuse=True)`.

In [121]:
# TODO: ALL Keras imports go here.

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
from tqdm import tqdm
from __future__ import print_function, division
import matplotlib.pyplot as plt
import sys
import numpy as np

import keras
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, UpSampling2D
from keras.layers.core import Reshape, Dense, Dropout, Flatten, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras import initializers
from keras import layers
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

from keras import backend as K 
# TODO Depending on the ordering of your kernels, and batches you might need to change this!
K.set_image_dim_ordering('th')

In [122]:
# Load MNIST data
def load_data(path='mnist.npz'):
    """Loads the MNIST dataset.
    # Arguments
        path: path where to cache the dataset locally
            (relative to ~/.keras/datasets).
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    path = "mnist.npz"
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()
    return (x_train, y_train), (x_test, y_test)

In [175]:
# Generator
def build_generator(latent_dim):
    # TODO. Make sure to use a tanh activation at the last layer and compile the model before returning it
    # For upsampling the image, you could use UpSampling2D function from Keras library. 
    # Use adam optimizer with: adam = Adam(0.0002, 0.5)
    adam = Adam(0.0002, 0.5)
    generator_input = keras.Input(shape=(latent_dim,))
    
    x = layers.Dense(128* 14 * 14)(generator_input)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((128,14,14))(x)
    
    x= layers.Conv2D(256,5, padding='same')(x)
    x= layers.LeakyReLU()(x)
    
    x= layers.Conv2DTranspose(256,4, strides=2, padding='same')(x)
    x= layers.LeakyReLU()(x)
    
    x = layers.Conv2D(256,5, padding='same')(x)
    x = layers.LeakyReLU()(x)
    x=  layers.Conv2D(256,5, padding='same')(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
    generator = keras.models.Model(generator_input,x)
    generator.summary() 
    """
    model = Sequential()
    dropout_prob = 0.4
    
    model.add(Dense(7*7*256, input_dim=100))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    model.add(Reshape((7,7,256)))
    model.add(Dropout(dropout_prob))
    
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    
    model.add(UpSampling2D())
    model.add(Conv2D(64, 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    
    model.add(Conv2D(32, 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    
    model.add(Conv2D(1, 5, padding='same'))
    model.add(Activation('sigmoid'))
    adam = Adam(0.0002, 0.5)
    """
    generator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return generator
    
    
    

In [186]:
# Discriminator
def build_discriminator(img_shape):
    # TODO. Make sure to use a sigmoid at the last layer and compile the model before returning it
    # Use adam optimizer with adam = Adam(0.0002, 0.5)
    discriminator_input = keras.Input(shape=img_shape)
    
    x= layers.Conv2D(128,3)(discriminator_input)
    x= layers.LeakyReLU()(x)
    x= layers.Conv2D(128,4,strides=2)(x)
    x= layers.LeakyReLU()(x)
    x= layers.Conv2D(128,4,strides=2)(x)
    x= layers.LeakyReLU()(x)
    x= layers.Conv2D(128,4,strides=2)(x)
    x= layers.Flatten()(x)
    x= layers.LeakyReLU()(x)
    x= layers.Dropout(0.4)(x)
    x =layers.Dense(1, activation='sigmoid')(x)
    discriminator = keras.models.Model(discriminator_input, x)
    discriminator.summary()
    """
    model = Sequential()
    #input_shape = (28, 28, 1)
    dropout_prob = 0.4

    model.add(Conv2D(64, 5, strides=2, input_shape=img_shape, padding='same'))
    model.add(LeakyReLU())
    
    model.add(Conv2D(128, 5, strides=2, padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(dropout_prob))
    
    model.add(Conv2D(256, 5, strides=2, padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(dropout_prob))
    
    model.add(Conv2D(512, 5, strides=1, padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(dropout_prob))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    """
    adam = Adam(0.0002, 0.5)
    discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return discriminator


In [187]:
##################### TODO #####################
# Now that we've built the disriminator and generator, lets combine the two for the full end to end system.
# This is where we are calculating D(G(z))!
# We will set up a Model object to train the generator to fool the discriminator. We need to turn of
# weight updates for the discriminator, create an Input object for the generator with the right
# dimension, run that through the generator, and run the output of the generator through the discriminator. 
# You should compile this new Model object.
def build_gan(discriminator, generator, latent_dim):
    adam = Adam(0.0002, 0.5)
    gan = Sequential()
    gan.add(generator)

    ## Disable layers in discriminator
    discriminator.trainable = False

    gan.add(discriminator)
    gan.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return gan

In [188]:
# Parameters for our mnist dataset. 
img_rows = 28
img_cols = 28
channels = 1
# Feel free to change the order of channels, rows, and cols if you go ahead with a different order 
# in your discriminator and generator. 
img_shape = (channels, img_rows, img_cols)
latent_dim = 100

In [189]:
# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, latent_dim])
    generatedImages = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i, 0], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/dcgan_generated_image_epoch_%d.png' % epoch)

In [192]:
# Build and compile the discriminator
discriminator = build_discriminator(img_shape)

# Build and compile the generator
generator = build_generator(latent_dim)

# Build and compile the combined network
gan = build_gan(discriminator, generator, latent_dim)


epochs = 10
batch_size = 128
save_interval = 50

# Load MNIST data and rescale -1 to 1
(X_train, y_train), (X_test, y_test) = load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train[:, np.newaxis, :, :]

batchCount = X_train.shape[0] / batch_size
for e in range(1, epochs+1):
    print('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(int(batchCount))):
        
        # Get a random sample from real images, and from random noise
        #noise = ...
        #noise =  np.random.randint(0, len(X_train) - batch/2)
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
        #imageBatch = ... 
        imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batch_size), :, :, :]

        # Generate fake MNIST images, from noise
        generatedImages = generator.predict(noise)
        
        # Lets train the discriminator first. 
        # We will concatenate the real images and fake images into a variable X
        #X = ...
        print(imageBatch.shape)
        print(generatedImages.shape)
        X = np.concatenate((imageBatch, generatedImages), axis = 0)

        # Create the labels for fake and real data, composed of 0s and 1s
        #yDis = ...
        yDis = np.ones([2*batch_size, 1])
        yDis[batch_size:, :] = 0 


        # Train discriminator
        discriminator.trainable = True
        dloss = discriminator.train_on_batch(X, yDis)

        # Now lets train the generator
        # Generate batch_size sized random noise 
        #noise = ...
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
        
        # Generate the labels for the generator
        #yGen = ...
        yGen = np.ones([batch_size, 1])
        discriminator.trainable = False
        gloss = gan.train_on_batch(noise, yGen)
        
    if e == 1 or e % 5 == 0:
        plotGeneratedImages(e)
        

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        (None, 1, 28, 28)         0         
_________________________________________________________________
conv2d_207 (Conv2D)          (None, 128, 26, 26)       1280      
_________________________________________________________________
leaky_re_lu_218 (LeakyReLU)  (None, 128, 26, 26)       0         
_________________________________________________________________
conv2d_208 (Conv2D)          (None, 128, 12, 12)       262272    
_________________________________________________________________
leaky_re_lu_219 (LeakyReLU)  (None, 128, 12, 12)       0         
_________________________________________________________________
conv2d_209 (Conv2D)          (None, 128, 5, 5)         262272    
_________________________________________________________________
leaky_re_lu_220 (LeakyReLU)  (None, 128, 5, 5)         0         
__________

  0%|          | 0/468 [00:00<?, ?it/s]

--------------- Epoch 1 ---------------
(128, 1, 28, 28)
(128, 1, 28, 28)


  0%|          | 1/468 [00:08<1:08:59,  8.86s/it]

(128, 1, 28, 28)
(128, 1, 28, 28)


...

100%|█████████▉| 466/468 [05:26<00:01,  1.43it/s]

(128, 1, 28, 28)
(128, 1, 28, 28)


100%|█████████▉| 467/468 [05:27<00:00,  1.43it/s]

(128, 1, 28, 28)
(128, 1, 28, 28)


100%|██████████| 468/468 [05:27<00:00,  1.43it/s]


## Results
Please embed some of the images you have generated and turn them in with your notebook!